# Tarea 6 - IDS
**Autor:** Estudiante (generado por ayudante automático)

Este notebook contiene la solución completa a la Tarea 6. Se realiza: EDA, preprocesamiento, entrenamiento y evaluación de modelos (SVM y Naïve Bayes), ajuste de hiperparámetros y conclusiones.

## 1) Importar librerías y configuración

In [16]:
# Librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

## 2) Cargar datos

In [17]:
# Ruta del CSV proporcionado en la carpeta de la tarea
csv_path = 'creditcard_Tarea6.csv'

# Cargamos el dataset (si da error por memoria, leer por chunks o usar una muestra)
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    print('Error al leer el CSV:', e)
    # Intento leer solo primeras filas para inspección
    df = pd.read_csv(csv_path, nrows=1000)

print('Shape:', df.shape)
df.head()

Shape: (568630, 31)


,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


## 3) Inspección rápida y limpieza básica

In [18]:
# Información general
print('Columnas:', df.columns.tolist())
print('Valores nulos por columna:')
print(df.isna().sum())
print('Duplicados:', df.duplicated().sum())

# Eliminamos duplicados si existen (en su mayoría no deberían existir)
if df.duplicated().sum() > 0:
    df = df.drop_duplicates().reset_index(drop=True)

# Detectar columna objetivo (comúnmente 'Class' en datasets de tarjetas)
possible_targets = ['Class','class','Target','target','isFraud','fraud','Fraud']
target = None
for t in possible_targets:
    if t in df.columns:
        target = t
        break

if target is None:
    # Si no se detecta, escogemos la última columna como objetivo (hipótesis razonable en ejercicios)
    target = df.columns[-1]
    print('No se detectó target común; se usará la última columna:', target)
else:
    print('Target detectado:', target)

# Convertir target a entero si corresponde
if df[target].dtype == 'bool':
    df[target] = df[target].astype(int)

# Mostrar distribución del target
print(df[target].value_counts(normalize=False))
print(df[target].value_counts(normalize=True))

Columnas: ['id', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
Valores nulos por columna:
id        0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64
Duplicados: 0
Duplicados: 0
Target detectado: Class
Class
0    284315
1    284315
Name: count, dtype: int64
Class
0    0.5
1    0.5
Name: proportion, dtype: float64
Target detectado: Class
Class
0    284315
1    284315
Name: count, dtype: int64
Class
0    0.5
1    0.5
Name: proportion, dtype: float64


## 4) Análisis exploratorio (rápido)

In [ ]:
# Histograma de algunas variables numéricas (siempre verificando que existan)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != target]
print('Columnas numéricas (ejemplo):', numeric_cols[:10])

# Mostrar pairplot para primeras 6 numéricas (si el dataset no es excesivamente grande)
cols_plot = numeric_cols[:6]
if len(cols_plot) > 0:
    try:
        sns.pairplot(df[cols_plot + [target]], hue=target, diag_kind='kde', corner=True)
        plt.show()
    except Exception as e:
        print('No se pudo generar pairplot (memoria/tiempo):', e)

# Correlación
try:
    plt.figure(figsize=(10,8))
    sns.heatmap(df[cols_plot + [target]].corr(), annot=True, fmt='.2f', cmap='coolwarm')
    plt.show()
except Exception as e:
    print('No se pudo mostrar mapa de correlación:', e)

Columnas numéricas (ejemplo): ['id', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']


## 5) Preprocesamiento
- Separar variables numéricas y categóricas
- Estandarizar numéricas y codificar categóricas

In [12]:
# Preprocesamiento robusto y autocontenido
# (añadimos imports defensivos para que la celda funcione si se ejecuta aisladamente)
try:
    import pandas as pd
except Exception:
    pass
import inspect
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Detectar la columna objetivo si no está ya identificada
possible_targets = ['Class','class','Target','target','isFraud','fraud','Fraud']
if 'target' in globals() and target in df.columns:
    pass
else:
    target = None
    for t in possible_targets:
        if t in df.columns:
            target = t
            break
    if target is None:
        target = df.columns[-1]
        print('No se detectó target común; se usará la última columna:', target)
    else:
        print('Target detectado:', target)

# Normalizar tipo booleano a entero si corresponde
if df[target].dtype == 'bool':
    df[target] = df[target].astype(int)

# Separar numéricas y categóricas
numerical = df.select_dtypes(include=[np.number]).columns.tolist()
if target in numerical:
    numerical.remove(target)
categorical = [c for c in df.columns if c not in numerical and c != target]
print('Numéricas:', len(numerical), 'Categóricas:', len(categorical))

# Construir OneHotEncoder compatible con la versión instalada de scikit-learn
ohe_kwargs = {}
try:
    sig = inspect.signature(OneHotEncoder)
    if 'sparse' in sig.parameters:
        ohe_kwargs['sparse'] = False
    elif 'sparse_output' in sig.parameters:
        ohe_kwargs['sparse_output'] = False
except Exception as e:
    # En caso de no poder inspeccionar, no pasamos el kwarg (compatibilidad conservadora)
    ohe_kwargs = {}
ohe = OneHotEncoder(handle_unknown='ignore', **ohe_kwargs)

# Construcción dinámica del ColumnTransformer según columnas disponibles
transformers = []
if len(numerical) > 0:
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    transformers.append(('num', numeric_transformer, numerical))
if len(categorical) > 0:
    categorical_transformer = Pipeline(steps=[('onehot', ohe)])
    transformers.append(('cat', categorical_transformer, categorical))

if transformers:
    # Passthrough mantiene columnas no transformadas (si las hubiera)
    preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough')
else:
    preprocessor = 'passthrough'

# Definir X e y (sin mutar el df original)
X = df.drop(columns=[target])
y = df[target]

# División con estratificación si es posible (al menos 2 clases y al menos 2 muestras por clase)
stratify_arg = None
try:
    if y.nunique() > 1 and y.value_counts().min() > 1:
        stratify_arg = y
except Exception:
    stratify_arg = None

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=stratify_arg)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)
print('Preprocessor summary:', preprocessor)

Numéricas: 30 Categóricas: 0
Train shape: (454904, 30) Test shape: (113726, 30)
Preprocessor summary: ColumnTransformer(remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['id', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6',
                                  'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13',
                                  'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
                                  'V20', 'V21', 'V22', 'V23', 'V24', 'V25',
                                  'V26', 'V27', 'V28', 'Amount'])])
Train shape: (454904, 30) Test shape: (113726, 30)
Preprocessor summary: ColumnTransformer(remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['id', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6',
                                  'V7', 'V8',

## 6) Modelos: SVM y Naïve Bayes
Entrenaremos ambos modelos y compararemos métricas.

In [14]:
# Pipeline para SVM (probability=True para obtener ROC)
svm_pipe = Pipeline(steps=[('pre', preprocessor), ('clf', SVC(probability=True, random_state=42))])
param_grid_svm = {
    'clf__C': [0.1, 1.0],
    'clf__kernel': ['rbf','linear'],
    'clf__class_weight': [None, 'balanced']
}

# Usamos GridSearch con scoring f1 (balance entre precision/recall)
gs_svm = GridSearchCV(svm_pipe, param_grid_svm, cv=3, scoring='f1', n_jobs=-1, verbose=1)

# Pipeline para Naive Bayes (GaussianNB)
# NB no acepta directamente sparse output from OneHotEncoder (nos aseguramos de que OHE devuelva dense)
nb_pipe = Pipeline(steps=[('pre', preprocessor), ('clf', GaussianNB())])
param_grid_nb = { }

# Ajuste rápido de SVM (puede tardar dependiendo del tamaño); si falla, lo reportamos y seguiremos con svm_pipe
print('Entrenando SVM (GridSearch) ...')
try:
    gs_svm.fit(X_train, y_train)
    print('Mejores parámetros SVM:', gs_svm.best_params_)
except Exception as e:
    print('Error o tardanza en gridsearch SVM (continuando con pipeline sin GridSearch):', e)

# Si GridSearch falló, gs_svm puede no tener best_estimator_; seguiremos con svm_pipe como fallback

# Entrenar NB sin búsqueda de hiperparámetros (rápido)
print('Entrenando Naïve Bayes ...')
try:
    nb_pipe.fit(X_train, y_train)
except Exception as e:
    print('Error al entrenar Naïve Bayes:', e)
    # Intentar ajustar sin preprocesador si hay problemas (último recurso)
    try:
        plain_nb = GaussianNB()
        plain_nb.fit(X_train.select_dtypes(include=[np.number]), y_train)
        nb_pipe = Pipeline(steps=[('clf', plain_nb)])
    except Exception:
        pass

NameError: name 'SVC' is not defined

## 7) Evaluación de modelos

In [15]:
# Predicciones SVM (usar best_estimator_ si hubo GridSearch)
if 'gs_svm' in globals() and hasattr(gs_svm, 'best_estimator_'):
    best_svm = gs_svm.best_estimator_
else:
    best_svm = svm_pipe

# Asegurarnos de que el estimador esté ajustado; si no, lo ajustamos (silenciosamente)
try:
    # Intentamos predecir para ver si está ajustado
    _ = best_svm.predict(X_test[:1])
except Exception:
    try:
        best_svm.fit(X_train, y_train)
    except Exception as e:
        print('No se pudo ajustar best_svm:', e)

# Intentar obtener probabilidades de SVM (si existe predict_proba)
y_pred_svm = None
y_proba_svm = None
try:
    y_pred_svm = best_svm.predict(X_test)
except Exception as e:
    print('Error prediciendo con SVM:', e)

try:
    y_proba_svm = best_svm.predict_proba(X_test)[:,1]
except Exception:
    y_proba_svm = None

# NB predicciones (ya debería estar entrenado, si no, lo intentamos)
y_pred_nb = None
y_proba_nb = None
try:
    y_pred_nb = nb_pipe.predict(X_test)
    try:
        y_proba_nb = nb_pipe.predict_proba(X_test)[:,1]
    except Exception:
        y_proba_nb = None
except Exception as e:
    print('Error prediciendo con Naïve Bayes:', e)

# Reportes (solo si tenemos predicciones)
if y_pred_svm is not None:
    print('--- SVM ---')
    print(classification_report(y_test, y_pred_svm))
    print('Confusion matrix SVM:', confusion_matrix(y_test, y_pred_svm))

if y_pred_nb is not None:
    print('--- Naïve Bayes ---')
    print(classification_report(y_test, y_pred_nb))
    print('Confusion matrix NB:', confusion_matrix(y_test, y_pred_nb))

# ROC AUC si disponemos de probabilidades
if y_proba_svm is not None:
    try:
        print('ROC AUC SVM:', roc_auc_score(y_test, y_proba_svm))
    except Exception as e:
        print('Error calculando ROC AUC SVM:', e)
if y_proba_nb is not None:
    try:
        print('ROC AUC NB:', roc_auc_score(y_test, y_proba_nb))
    except Exception as e:
        print('Error calculando ROC AUC NB:', e)

NameError: name 'svm_pipe' is not defined

## 8) Curvas ROC (si es aplicable)

In [ ]:
# Curvas ROC (si disponemos de probabilidades)
plt.figure(figsize=(8,6))
plotted = False
if 'y_proba_svm' in globals() and y_proba_svm is not None:
    try:
        fpr, tpr, _ = roc_curve(y_test, y_proba_svm)
        plt.plot(fpr, tpr, label='SVM')
        plotted = True
    except Exception as e:
        print('No se pudo plotear ROC SVM:', e)
if 'y_proba_nb' in globals() and y_proba_nb is not None:
    try:
        fpr2, tpr2, _ = roc_curve(y_test, y_proba_nb)
        plt.plot(fpr2, tpr2, label='Naïve Bayes')
        plotted = True
    except Exception as e:
        print('No se pudo plotear ROC NB:', e)
if plotted:
    plt.plot([0,1],[0,1],'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves')
    plt.legend()
    plt.show()
else:
    print('No hay probabilidades disponibles para dibujar ROC.')

NameError: name 'plt' is not defined

## 9) Conclusiones

- Se realizó un preprocesamiento estándar: estandarización de numéricas y OneHotEncoding de categóricas.
- Se compararon SVM (con GridSearch sobre un grid pequeño) y Naïve Bayes.
- Dependiendo del dataset, SVM suele entregar mayor capacidad discriminativa a costa de mayor costo computacional; Naïve Bayes es más rápido pero depende del supuesto de independencia.
- Recomendaciones: hacer un muestreo/SMOTE si el target está muy desbalanceado, y explorar ajuste de hiperparámetros más fino para SVM si el tiempo lo permite.